In [55]:
#importamos librerias básicas
import pandas as pd #manipulación dataframes
import numpy as np #matrices y vectores
import matplotlib.pyplot as plt #graficación

In [56]:
data=pd.read_excel("./data/Base de datos UBER.xlsx")
data.head() #Listar las primeras filas de data

,Date,Time,Booking Status,Avg VTAT,Avg CTAT,Cancelled Rides by Customer,Reason for cancelling by Customer,Cancelled Rides by Driver,Driver Cancellation Reason,Incomplete Rides,Incomplete Rides Reason,Booking Value,Ride Distance,Payment Method
0,2024-03-23,12:29:38,No Driver Found,0.0,0.0,No,No Report,No,No Report,No,No Report,0,0.00,No Report
1,2024-11-29,18:01:39,Incomplete,4.9,0.0,No,No Report,No,No Report,Si,Vehicle Breakdown,237,5.73,UPI
2,2024-08-23,08:56:10,Completed,13.4,0.0,No,No Report,No,No Report,No,No Report,627,13.58,Debit Card
3,2024-10-21,17:17:25,Completed,13.1,0.0,No,No Report,No,No Report,No,No Report,416,34.02,UPI
4,2024-09-16,22:08:00,Completed,5.3,0.0,No,No Report,No,No Report,No,No Report,737,48.21,UPI


In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 14 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   Date                               150000 non-null  datetime64[ns]
 1   Time                               150000 non-null  object        
 2   Booking Status                     150000 non-null  object        
 3   Avg VTAT                           150000 non-null  float64       
 4   Avg CTAT                           150000 non-null  float64       
 5   Cancelled Rides by Customer        150000 non-null  object        
 6   Reason for cancelling by Customer  150000 non-null  object        
 7   Cancelled Rides by Driver          150000 non-null  object        
 8   Driver Cancellation Reason         150000 non-null  object        
 9   Incomplete Rides                   150000 non-null  object        
 10  Incomplete Rides Rea

In [58]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Re-create the 'DateTime' column by combining 'Date' and 'Time'
data['DateTime'] = pd.to_datetime(data['Date'].astype(str) + ' ' + data['Time'].astype(str), errors='coerce')

# Drop the original 'Date' and 'Time' columns
data = data.drop(['Date', 'Time'], axis=1)


# --- 1. Convertir DateTime ---
# The 'DateTime' column already exists and is in datetime format
data["fecha"] = data["DateTime"]
data["hora"] = data["DateTime"].dt.hour

# --- 2. Extraer características de fecha ---
data["anio"] = data["fecha"].dt.year
data["mes"] = data["fecha"].dt.month
data["dia_semana"] = data["fecha"].dt.dayofweek   # 0=lunes, 6=domingo

# --- 3. Codificar hora de manera cíclica ---
data["hora_sin"] = np.sin(2 * np.pi * data["hora"] / 24)
data["hora_cos"] = np.cos(2 * np.pi * data["hora"] / 24)

# --- 4. (Opcional) Discretizar hora en franjas horarias ---
def discretizar_hora(hora):
    if 5 <= hora < 12:
        return "mañana"
    elif 12 <= hora < 18:
        return "tarde"
    elif 18 <= hora < 24:
        return "noche"
    else:
        return "madrugada"

data["franja_horaria"] = data["hora"].apply(discretizar_hora)

# Convertir franja_horaria en variables dummy
data = pd.get_dummies(data, columns=["franja_horaria"], dtype=int)

In [59]:
# Función para asignar estación según mes
def asignar_estacion(mes):
    if mes in [12, 1, 2]:
        return "Invierno"
    elif mes in [3, 4, 5]:
        return "Verano"
    elif mes in [6, 7, 8, 9]:
        return "Monzon"
    else:  # 10, 11
        return "Post-Monzon"

# Crear columna de estación
data["estacion"] = data["mes"].map(asignar_estacion)

# Crear variables dummies sin borrar nada más
data = pd.get_dummies(data, columns=["estacion"], drop_first=False, dtype=int)

In [60]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 28 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   Booking Status                     150000 non-null  object        
 1   Avg VTAT                           150000 non-null  float64       
 2   Avg CTAT                           150000 non-null  float64       
 3   Cancelled Rides by Customer        150000 non-null  object        
 4   Reason for cancelling by Customer  150000 non-null  object        
 5   Cancelled Rides by Driver          150000 non-null  object        
 6   Driver Cancellation Reason         150000 non-null  object        
 7   Incomplete Rides                   150000 non-null  object        
 8   Incomplete Rides Reason            150000 non-null  object        
 9   Booking Value                      150000 non-null  int64         
 10  Ride Distance       

In [61]:
#Eliminar una columna
data = data.drop(['DateTime','fecha','hora','anio','mes','dia_semana','hora_cos','hora_sin'], axis=1)
data.head()

,Booking Status,Avg VTAT,Avg CTAT,Cancelled Rides by Customer,Reason for cancelling by Customer,Cancelled Rides by Driver,Driver Cancellation Reason,Incomplete Rides,Incomplete Rides Reason,Booking Value,Ride Distance,Payment Method,franja_horaria_madrugada,franja_horaria_mañana,franja_horaria_noche,franja_horaria_tarde,estacion_Invierno,estacion_Monzon,estacion_Post-Monzon,estacion_Verano
0,No Driver Found,0.0,0.0,No,No Report,No,No Report,No,No Report,0,0.00,No Report,0,0,0,1,0,0,0,1
1,Incomplete,4.9,0.0,No,No Report,No,No Report,Si,Vehicle Breakdown,237,5.73,UPI,0,0,1,0,0,0,1,0
2,Completed,13.4,0.0,No,No Report,No,No Report,No,No Report,627,13.58,Debit Card,0,1,0,0,0,1,0,0
3,Completed,13.1,0.0,No,No Report,No,No Report,No,No Report,416,34.02,UPI,0,0,0,1,0,0,1,0
4,Completed,5.3,0.0,No,No Report,No,No Report,No,No Report,737,48.21,UPI,0,0,1,0,0,1,0,0


In [62]:
#Corrección de variables categóricas
data['Vehicle Type']=data['Vehicle Type'].astype('category')
data['Booking Status']=data['Booking Status'].astype('category')
data['Cancelled Rides by Customer']=data['Cancelled Rides by Customer'].astype('category')
data['Reason for cancelling by Customer']=data['Reason for cancelling by Customer'].astype('category')
data['Cancelled Rides by Driver']=data['Cancelled Rides by Driver'].astype('category')
data['Driver Cancellation Reason']=data['Driver Cancellation Reason'].astype('category')
data['Incomplete Rides']=data['Incomplete Rides'].astype('category')
data['Incomplete Rides Reason']=data['Incomplete Rides Reason'].astype('category')
data['Payment Method']=data['Payment Method'].astype('category')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 20 columns):
 #   Column                             Non-Null Count   Dtype   
---  ------                             --------------   -----   
 0   Booking Status                     150000 non-null  category
 1   Avg VTAT                           150000 non-null  float64 
 2   Avg CTAT                           150000 non-null  float64 
 3   Cancelled Rides by Customer        150000 non-null  category
 4   Reason for cancelling by Customer  150000 non-null  category
 5   Cancelled Rides by Driver          150000 non-null  category
 6   Driver Cancellation Reason         150000 non-null  category
 7   Incomplete Rides                   150000 non-null  category
 8   Incomplete Rides Reason            150000 non-null  category
 9   Booking Value                      150000 non-null  int64   
 10  Ride Distance                      150000 non-null  float64 
 11  Payment Method            

In [63]:
#Descripción de variables numéricas (para mirar si hay problemas de calidad de datos)
data.describe()

,Avg VTAT,Avg CTAT,Booking Value,Ride Distance,franja_horaria_madrugada,franja_horaria_mañana,franja_horaria_noche,franja_horaria_tarde,estacion_Invierno,estacion_Monzon,estacion_Post-Monzon,estacion_Verano
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000
mean,7.864407,18.431355,345.641220,16.753168,0.045173,0.303053,0.329200,0.322573,0.246920,0.334807,0.166967,0.251307
std,4.230640,15.737330,403.423487,16.291118,0.207685,0.459580,0.469924,0.467463,0.431221,0.471924,0.372947,0.433766
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.800000,20.400000,244.000000,13.060000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,11.000000,32.000000,521.000000,30.650000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000
max,20.000000,45.000000,4277.000000,50.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [64]:
#Descripción variables categóricas
data['Vehicle Type'].value_counts().plot(kind='bar')

In [65]:
data['Booking Status'].value_counts().plot(kind='pie', autopct='%.0f%%')

<Axes: ylabel='count'>

In [66]:
# Cargar librería para Profiling
from ydata_profiling import ProfileReport

In [67]:
profile_data=ProfileReport(data, minimal=False) # minimal=True
profile_data

Render HTML: 100%|██████████| 1/1 [00:00<00:00, 13.74it/s]


In [68]:
#Guardamos en html el perfilado de datos (para poder compartirlo como un informe en cualquier máquina)
profile_data.to_file(output_file="output.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 439.61it/s]


In [69]:
#Se crean dummies a las variables predictoras categóricas (no a la variable obj)
data = pd.get_dummies(data, columns=['Vehicle Type','Reason for cancelling by Customer','Cancelled Rides by Driver','Driver Cancellation Reason','Incomplete Rides','Incomplete Rides Reason','Payment Method','Cancelled Rides by Customer'], drop_first=False, dtype=int)
data.head()

KeyError: "['Vehicle Type'] not in index"

In [ ]:
#Se codifican las categorias de la VARIABLE OBJETIVO

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
data["Booking Status"]=labelencoder.fit_transform(data["Booking Status"]) #Objetivo

data.head()

In [ ]:
#División 70-30
from sklearn.model_selection import train_test_split
X = data.drop("Booking Status", axis = 1) # Variables predictoras
Y = data['Booking Status'] #Variable objetivo
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, stratify=Y) #Muestreo estratificado (mantiene la proporción de clases de la variable objetivo al dividr datos)
Y_train.value_counts().plot(kind='bar')

In [ ]:
#Variable objetivo del 30%
Y_test.value_counts().plot(kind='bar')

In [ ]:
X_train.head()

In [ ]:
#Creación del modelo con el conjunto de entrenamiento
from sklearn.tree import DecisionTreeClassifier

modelTree = DecisionTreeClassifier(criterion='gini', min_samples_leaf=2, max_depth=None) #gini, entropy
modelTree.fit(X_train, Y_train) #70% train


In [ ]:
from sklearn.tree import plot_tree
plt.figure(figsize=(6,8))
plot_tree(modelTree, feature_names=X_train.columns.values, class_names=labelencoder.classes_, rounded=True, filled=True)
plt.show()

In [ ]:
#Evaluación 30%
from sklearn import metrics

Y_pred = modelTree.predict(X_test) #30% Test
print(Y_pred)


In [ ]:
#Exactitud
exactitud=metrics.accuracy_score(y_true=Y_test, y_pred=Y_pred)
print(exactitud)

In [ ]:
#Matriz de confusion
from sklearn import metrics

cm=metrics.confusion_matrix(y_true=Y_test, y_pred=Y_pred)
cm

In [ ]:
#Plot de la matriz de confusion
disp=metrics.ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=labelencoder.classes_)
disp.plot()

In [ ]:
#Precision, Recall, f1, exactitud
print(metrics.classification_report( y_true=Y_test, y_pred=Y_pred, target_names=labelencoder.classes_))

In [ ]:
#Normalizacion las variables numéricas (las dummies no se normalizan)
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
min_max_scaler.fit(data[['Avg VTAT']]) #Ajuste de los parametros: max - min

#Se aplica la normalización a 70%  y 30%
X_train[['Avg VTAT']]= min_max_scaler.transform(X_train[['Avg VTAT']]) #70%
X_test[['Avg VTAT']]= min_max_scaler.transform(X_test[['Avg VTAT']]) #30%
#X_train[['Avg CTAT']]= min_max_scaler.transform(X_train[['Avg CTAT']]) #70%
#X_test[['Avg CTAT']]= min_max_scaler.transform(X_test[['Avg CTAT']]) #30%
#X_train[['Booking Value']]= min_max_scaler.transform(X_train[['Booking Value']]) #70%
#X_test[['Booking Value']]= min_max_scaler.transform(X_test[['Booking Value']]) #30%
#X_train[['Ride Distance']]= min_max_scaler.transform(X_train[['Ride Distance']]) #70%
#X_test[['Ride Distance']]= min_max_scaler.transform(X_test[['Ride Distance']]) #30%
#X_train.head()

In [ ]:
#Normalizacion las variables numéricas (las dummies no se normalizan)
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
min_max_scaler.fit(data[['Avg CTAT']]) #Ajuste de los parametros: max - min

#Se aplica la normalización a 70%  y 30%
#X_train[['Avg VTAT']]= min_max_scaler.transform(X_train[['Avg VTAT']]) #70%
#X_test[['Avg VTAT']]= min_max_scaler.transform(X_test[['Avg VTAT']]) #30%
X_train[['Avg CTAT']]= min_max_scaler.transform(X_train[['Avg CTAT']]) #70%
X_test[['Avg CTAT']]= min_max_scaler.transform(X_test[['Avg CTAT']]) #30%
#X_train[['Booking Value']]= min_max_scaler.transform(X_train[['Booking Value']]) #70%
#X_test[['Booking Value']]= min_max_scaler.transform(X_test[['Booking Value']]) #30%
#X_train[['Ride Distance']]= min_max_scaler.transform(X_train[['Ride Distance']]) #70%
#X_test[['Ride Distance']]= min_max_scaler.transform(X_test[['Ride Distance']]) #30%
#X_train.head()

In [ ]:
#Normalizacion las variables numéricas (las dummies no se normalizan)
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
min_max_scaler.fit(data[['Booking Value']]) #Ajuste de los parametros: max - min

#Se aplica la normalización a 70%  y 30%
#X_train[['Avg VTAT']]= min_max_scaler.transform(X_train[['Avg VTAT']]) #70%
#X_test[['Avg VTAT']]= min_max_scaler.transform(X_test[['Avg VTAT']]) #30%
#X_train[['Avg CTAT']]= min_max_scaler.transform(X_train[['Avg CTAT']]) #70%
#X_test[['Avg CTAT']]= min_max_scaler.transform(X_test[['Avg CTAT']]) #30%
X_train[['Booking Value']]= min_max_scaler.transform(X_train[['Booking Value']]) #70%
X_test[['Booking Value']]= min_max_scaler.transform(X_test[['Booking Value']]) #30%
#X_train[['Ride Distance']]= min_max_scaler.transform(X_train[['Ride Distance']]) #70%
#X_test[['Ride Distance']]= min_max_scaler.transform(X_test[['Ride Distance']]) #30%
#X_train.head()

In [ ]:
#Normalizacion las variables numéricas (las dummies no se normalizan)
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
min_max_scaler.fit(data[['Ride Distance']]) #Ajuste de los parametros: max - min

#Se aplica la normalización a 70%  y 30%
#X_train[['Avg VTAT']]= min_max_scaler.transform(X_train[['Avg VTAT']]) #70%
#X_test[['Avg VTAT']]= min_max_scaler.transform(X_test[['Avg VTAT']]) #30%
#X_train[['Avg CTAT']]= min_max_scaler.transform(X_train[['Avg CTAT']]) #70%
#X_test[['Avg CTAT']]= min_max_scaler.transform(X_test[['Avg CTAT']]) #30%
#X_train[['Booking Value']]= min_max_scaler.transform(X_train[['Booking Value']]) #70%
#X_test[['Booking Value']]= min_max_scaler.transform(X_test[['Booking Value']]) #30%
X_train[['Ride Distance']]= min_max_scaler.transform(X_train[['Ride Distance']]) #70%
X_test[['Ride Distance']]= min_max_scaler.transform(X_test[['Ride Distance']]) #30%
#X_train.head()

In [ ]:
#Aprendizaje KNN con 70%
from sklearn.neighbors  import KNeighborsClassifier #KNeighborsRegressor

modelKnn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')#euclidean, minkowski
modelKnn.fit(X_train, Y_train) #70%

In [ ]:
#Evaluación de Knn con 30%
from sklearn import metrics

Y_pred = modelKnn.predict(X_test) #30%

#Matriz de confusion
cm=metrics.confusion_matrix(y_true=Y_test, y_pred=Y_pred)
disp=metrics.ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=labelencoder.classes_)
disp.plot()

#Precision, Recall, f1, exactitud
print(metrics.classification_report( y_true=Y_test, y_pred=Y_pred, target_names=labelencoder.classes_))

# Curva ROC - Removed as this is for binary classification, not multi-class
# metrics.RocCurveDisplay.from_estimator(modelKnn,X_test, Y_test)

In [ ]:
#Red Neuronal

from sklearn.neural_network import MLPClassifier #MLPRegressor

#Solo se configura capas ocultas, no se configura capa de entrada y de salida
#activation -> función activación de la oculta
#hidden_layer_sizes=5,7 -> dos capas ocultas con 5 neuronas y 7 neuronas
#learning_rate-> tamaño del paso constante o decreciente
#learning_rate_init-> valor tasa de aprendizaje
#momentum->
#max_iter-> iteaciones
#random_state-> semilla para generacion numeros seudoaletorios
modelNN = MLPClassifier(activation="logistic",hidden_layer_sizes=(5), learning_rate='constant',
                     learning_rate_init=0.2, momentum= 0.3, max_iter=500, random_state=3)

modelNN.fit(X_train, Y_train) #70% normalizados

In [ ]:
#Loss es la desviación entre Y_train y el Y_pred
#loss_curve_ es una lista con los valores de la función de pérdida (loss) en cada iteración/época durante el entrenamiento de la red neuronal.
loss_values = modelNN.loss_curve_
plt.plot(loss_values)

In [ ]:
#Evaluación de Red Neuronal
from sklearn import metrics

Y_pred = modelNN.predict(X_test) #30%

#Matriz de confusion
cm=metrics.confusion_matrix(y_true=Y_test, y_pred=Y_pred)
disp=metrics.ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=labelencoder.classes_)
disp.plot()

#Precision, Recall, f1, exactitud
print(metrics.classification_report( y_true=Y_test, y_pred=Y_pred, target_names=labelencoder.classes_))

# Curva ROC - Removed as this is for binary classification, not multi-class
# metrics.RocCurveDisplay.from_estimator(modelNN,X_test, Y_test)

In [ ]:
#SVM
from sklearn.svm import SVC # SVR

modelSVM = SVC(kernel='linear') #'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'
modelSVM.fit(X_train, Y_train) #70%


In [ ]:
#Evaluación de SVM
from sklearn import metrics

Y_pred = modelSVM.predict(X_test) #30%

#Matriz de confusion
cm=metrics.confusion_matrix(y_true=Y_test, y_pred=Y_pred)
disp=metrics.ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=labelencoder.classes_)
disp.plot()

#Precision, Recall, f1, exactitud
print(metrics.classification_report( y_true=Y_test, y_pred=Y_pred, target_names=labelencoder.classes_))

# Curva ROC - Removed as this is for binary classification, not multi-class
# metrics.RocCurveDisplay.from_estimator(modelSVM,X_test, Y_test)

In [ ]:
#pickle es una librería de Python para serializar objetos (guardarlos en un archivo en formato binario)

import pickle
filename = 'modelo.pkl'
variables= X.columns._values
pickle.dump([modelTree,modelKnn, modelNN, modelSVM, labelencoder,variables,min_max_scaler], open(filename, 'wb'))